In [21]:
#encoding=utf-8
import jieba
import matplotlib.pyplot as plt
import numpy as np
import math
import re

Class=list()
intr=list()

def replace_all_blank(value):        #去掉標點符號
  result = re.sub('\W+', ' ', value).replace("_",  '')
  return result

#讀檔
def Read():
    with open('NLP_HW02.csv', 'r', newline='', encoding='utf-8') as csvfile:
        rows = csv.DictReader(csvfile)
        for row in rows:
          if row['Column3']=="劇情介紹":
              Class.append(row['Column2'])
              intr.append(row['Column3'])
          strin=' '.join(jieba.cut(replace_all_blank(row['Column3'])))
          row['Column2'] = row['Column2'].split(' ')[0]
          if row['Column2']!="NA":
            Class.append(row['Column2'])
            intr.append(strin)

Read()


In [24]:
import pandas as pd
import csv

#寫檔 只寫進分類和內容
def write():
    for i in range(len(Class)):
      with open('NLP_HW03_copy.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([  Class[i], intr[i]])

write()

In [29]:
#讀出整理好的檔案內容
df = pd.read_csv('NLP_HW03_copy.csv')
col = ['分類', '劇情介紹'] 
df = df[col] 
df = df[pd.notnull(df['劇情介紹'])] 
df.columns = ['分類', '劇情介紹'] 
df['category_id'] = df['分類'].factorize()[0] 
category_id_df = df[['分類', 'category_id']].drop_duplicates().sort_values('category_id') 
category_to_id = dict(category_id_df.values) 
id_to_category = dict(category_id_df[['category_id', '分類']].values)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='UTF-8', ngram_range=(1, 2), stop_words='english') 
features = tfidf.fit_transform(df.劇情介紹).toarray() 
labels = df.category_id 
features.shape

n = 20          
svd = TruncatedSVD(n)               
Features = svd.fit_transform(features)


In [45]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer 

X_train, X_test, y_train, y_test = train_test_split(Features, df['分類'], test_size=500, random_state = 100)
clf=KNeighborsClassifier(n_neighbors=31,p=2,weights="distance",algorithm='brute')
clf.fit(X_train,y_train)
print("正確率:",clf.score(X_test, y_test)*100,"%")

正確率: 58.4 %
